In [1]:
import json
from pyspark.sql import SparkSession, Row
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/19 20:00:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/19 20:00:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
pwd()

'/home/hx152/Project/tweet_project/hx/Tweet-database-project'

In [5]:
# data = spark.read.text("hdfs:/data/corona-out-3", lineSep="\r\n\n")

In [6]:
# data = spark.read.text("file:/home/hx152/Project/tweet_project/Dataset/corona-out-3", lineSep="\r\n\n")

In [35]:
data = spark.read.text("file:/home/hx152/Project/tweet_project/Dataset/test_data", lineSep="\n\n")

In [36]:
a = data.take(1)
a

[Row(value='{"created_at":"Sun Apr 12 18:27:25 +0000 2020","id":1249403767180668930,"id_str":"1249403767180668930","text":"RT @nuffsaidny: wishing death on people is weirdo behavior.","source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/iphone\\" rel=\\"nofollow\\"\\u003eTwitter for iPhone\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1242817830946508801,"id_str":"1242817830946508801","name":"juwelz v","screen_name":"juwelz_v","location":"Lower East Side, Manhattan","url":"https:\\/\\/linktr.ee\\/juwelzv","description":"Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production\\ud83c\\udfa5\\ud83d\\udcf8 #BringNyCultureBack \\ud83c\\udf0a\\ud83c\\udf0a\\ud83c\\udf0a !!","translator_type":"none","protected":false,"verified":false,"followers_count":43,"friends_count":118,"li

# Json process

In [37]:
a[0][0]

'{"created_at":"Sun Apr 12 18:27:25 +0000 2020","id":1249403767180668930,"id_str":"1249403767180668930","text":"RT @nuffsaidny: wishing death on people is weirdo behavior.","source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/iphone\\" rel=\\"nofollow\\"\\u003eTwitter for iPhone\\u003c\\/a\\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1242817830946508801,"id_str":"1242817830946508801","name":"juwelz v","screen_name":"juwelz_v","location":"Lower East Side, Manhattan","url":"https:\\/\\/linktr.ee\\/juwelzv","description":"Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production\\ud83c\\udfa5\\ud83d\\udcf8 #BringNyCultureBack \\ud83c\\udf0a\\ud83c\\udf0a\\ud83c\\udf0a !!","translator_type":"none","protected":false,"verified":false,"followers_count":43,"friends_count":118,"listed_count"

In [38]:
tweet_data =  json.loads(a[0][0])

In [39]:
tweet_data

{'created_at': 'Sun Apr 12 18:27:25 +0000 2020',
 'id': 1249403767180668930,
 'id_str': '1249403767180668930',
 'text': 'RT @nuffsaidny: wishing death on people is weirdo behavior.',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1242817830946508801,
  'id_str': '1242817830946508801',
  'name': 'juwelz v',
  'screen_name': 'juwelz_v',
  'location': 'Lower East Side, Manhattan',
  'url': 'https://linktr.ee/juwelzv',
  'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 43,
  'friends_count': 118,
  'listed_count': 0,
  'favourites_count': 722,


In [40]:
tweet_id = tweet_data['id']
usr_id = tweet_data['user']['id']
comment_id = tweet_data['in_reply_to_status_id']

# userid_tweetid_dict

In [41]:
def get_userid_and_tweetid(tweet_data):
    tweet_data =  json.loads(tweet_data[0])
    tweet_id = tweet_data['id']
    user_id = tweet_data['user']['id']
    return (user_id,[tweet_id])

In [42]:
a = data.rdd.map(get_userid_and_tweetid)

In [43]:
a.collect()

[(1242817830946508801, [1249403767180668930]),
 (1225145123920588805, [1249403768023678982]),
 (101007632, [1249403769193779202]),
 (1230170166614482944, [1249403769567227906]),
 (4707764075, [1249403770435493888]),
 (988174833849634816, [1249403770360016896])]

In [44]:
userid_tweetid_dict = a.reduceByKey(lambda v1,v2:v1+v2).collectAsMap()

In [45]:
userid_tweetid_dict

{1242817830946508801: [1249403767180668930],
 1225145123920588805: [1249403768023678982],
 101007632: [1249403769193779202],
 1230170166614482944: [1249403769567227906],
 4707764075: [1249403770435493888],
 988174833849634816: [1249403770360016896]}

# tweetid_quota_id_dict